#**Desafío 11 - Clasificación desde la Econometría**

##Francisca Pinto, 07 de septiembre de 2021

In [ ]:
import pandas as pd
import numpy as np

import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import linear_model

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
path_colab = "/content/southafricanheart.csv"
df = pd.read_csv(path_colab)
df.drop(columns = "Unnamed: 0", inplace = True)
df.head()

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1


##Descripción

En esta sesión trabajaremos el dataset south african heart, el cual contiene las siguientes variables:

1. sbp​: Presión Sanguínea Sistólica.
2. tobacco​: Promedio tabaco consumido por día.
3. ldl​: Lipoproteína de baja densidad.
4. adiposity​: Adiposidad.
5. famhist​: Antecedentes familiares de enfermedades cardiácas. (Binaria)
6. typea​: Personalidad tipo A
7. obesity​: Obesidad.
8. alcohol​: Consumo actual de alcohol.
9. age​: edad.
10. chd​: Enfermedad coronaria. (dummy)

##Desafío 1: Preparar el ambiente de trabajo

1. Cargue las librerías básicas para importación y manipulación de datos (numpy,
pandas), gráficos (matplotlib y seaborn) y de modelación econométrica
(statsmodels).

2. Importe el archivo southafricanheart.csv que se encuentra dentro del material de apoyo.

3. Realice una descripción del set importado mostrando:

- Lista con los nombres de variables importadas
- Un análisis descriptivo mediante ​.describe()
- Distribución de categorías para las variables ​famhist​ ​y ​chd.

In [ ]:
variables = list(df.columns)
variables

['sbp',
 'tobacco',
 'ldl',
 'adiposity',
 'famhist',
 'typea',
 'obesity',
 'alcohol',
 'age',
 'chd']

In [ ]:
df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   sbp        462 non-null    int64  
 1   tobacco    462 non-null    float64
 2   ldl        462 non-null    float64
 3   adiposity  462 non-null    float64
 4   famhist    462 non-null    object 
 5   typea      462 non-null    int64  
 6   obesity    462 non-null    float64
 7   alcohol    462 non-null    float64
 8   age        462 non-null    int64  
 9   chd        462 non-null    int64  
dtypes: float64(5), int64(4), object(1)
memory usage: 36.2+ KB


In [ ]:
df.describe()

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,chd
count,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000
mean,138.326840,3.635649,4.740325,25.406732,53.103896,26.044113,17.044394,42.816017,0.346320
std,20.496317,4.593024,2.070909,7.780699,9.817534,4.213680,24.481059,14.608956,0.476313
min,101.000000,0.000000,0.980000,6.740000,13.000000,14.700000,0.000000,15.000000,0.000000
25%,124.000000,0.052500,3.282500,19.775000,47.000000,22.985000,0.510000,31.000000,0.000000
50%,134.000000,2.000000,4.340000,26.115000,53.000000,25.805000,7.510000,45.000000,0.000000
75%,148.000000,5.500000,5.790000,31.227500,60.000000,28.497500,23.892500,55.000000,1.000000
max,218.000000,31.200000,15.330000,42.490000,78.000000,46.580000,147.190000,64.000000,1.000000


In [ ]:
def describe_var(dataframe):
  
  tmp = dataframe

  for index in tmp:

    if len(tmp[index].value_counts()) > 2:
      print(tmp[index].describe(), "\n")
    
    else:
      print(tmp[index].value_counts("%"), "\n")

describe_var(df)

count    462.000000
mean     138.326840
std       20.496317
min      101.000000
25%      124.000000
50%      134.000000
75%      148.000000
max      218.000000
Name: sbp, dtype: float64 

count    462.000000
mean       3.635649
std        4.593024
min        0.000000
25%        0.052500
50%        2.000000
75%        5.500000
max       31.200000
Name: tobacco, dtype: float64 

count    462.000000
mean       4.740325
std        2.070909
min        0.980000
25%        3.282500
50%        4.340000
75%        5.790000
max       15.330000
Name: ldl, dtype: float64 

count    462.000000
mean      25.406732
std        7.780699
min        6.740000
25%       19.775000
50%       26.115000
75%       31.227500
max       42.490000
Name: adiposity, dtype: float64 

Absent     0.584416
Present    0.415584
Name: famhist, dtype: float64 

count    462.000000
mean      53.103896
std        9.817534
min       13.000000
25%       47.000000
50%       53.000000
75%       60.000000
max       78.000000
Name: 

Los procedimientos anteriores se usan como forma de corroboración, el dataset no tiene datos nulos: en todas las variables tienen la misma cantidad de datos. Se tienen 462 filas en total.

Respecto a las variables binarias, se observa desbalance en la cantidad que existe por cada valor (0 y 1).

##Desafío 2

A continuación se presenta un modelo a estimar, considerando la presencia de enfermedad coronaria en el paciente (chd = 1) en un modelo de regresión logarítmica, considerando como variable independiente a "famhist".

Para ello ejecute los siguientes pasos:

1. Recodifique ​famhist​ a dummy, asignando 1 a la categoría minoritaria.
2. Utilice ​smf.logit​ para estimar el modelo.
3. Implemente una función ​inverse_logit que realice el mapeo de log-odds a
probabilidad.
4. Con el modelo estimado, responda lo siguiente:

- ¿Cuál es la probabilidad de un individuo con antecedentes familiares de tener
una enfermedad coronaria?
- ¿Cuál es la probabilidad de un individuo sin antecedentes familiares de tener
una enfermedad coronaria?
- ¿Cuál es la diferencia en la probabilidad entre un individuo con antecedentes
y otro sin antecedentes?
- Replique el modelo con ​smf.ols y comente las similitudes entre los
coeficientes estimados.

​Tip:​ Utilice β/4

Se genera nuevo dataframe con variable "famhist" binarizada con el método get_dummies: se utilizará "famhist_Present" para el análisis, es la variable con menor presencia (en comparación a Absent).

In [ ]:
df_bin = pd.get_dummies(data = df, columns = ["famhist"], drop_first = False)
df_bin = df_bin[["sbp", "tobacco", "ldl", "adiposity", "typea", "obesity", "alcohol", "age", "famhist_Absent", "famhist_Present", "chd"]] #para modificar el orden y dejar el vector objetivo como última columna
df_bin

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,famhist_Absent,famhist_Present,chd
0,160,12.00,5.73,23.11,49,25.30,97.20,52,0,1,1
1,144,0.01,4.41,28.61,55,28.87,2.06,63,1,0,1
2,118,0.08,3.48,32.28,52,29.14,3.81,46,0,1,0
3,170,7.50,6.41,38.03,51,31.99,24.26,58,0,1,1
4,134,13.60,3.50,27.78,60,25.99,57.34,49,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
457,214,0.40,5.98,31.72,64,28.45,0.00,58,1,0,0
458,182,4.20,4.41,32.10,52,28.61,18.72,52,1,0,1
459,108,3.00,1.59,15.23,40,20.09,26.64,55,1,0,0
460,118,5.40,11.61,30.79,64,27.35,23.97,40,1,0,0


1. El promedio de "famhist_Present" es 1, así como el de "famhist_Absent" es 0.
2. Luego se estima el log-odd, considerando estas medias.
3. Posteriormente, se usa la función logit_inv para calcular la probabilidad de ocurrencia de la enfermedad coronaria, considerando cada alternativa, para responder las preguntas.

In [ ]:
"""
-----
Función logit_inv
-----------------

Argumentos:

log_odd: Logaritmo de chance de ocurrencia (logg-odd) del cual se desea conocer el logaritmo inverso.

-----

Pasos:

La función calcula el logaritmo inverso de la cifra entregada, utilizando el método de Numpy .exp().

-----
"""

def logit_inv(log_odd):

  return 1 / (1 + np.exp( - log_odd ))

In [ ]:
model1 = smf.logit("chd ~ famhist_Present", data = df_bin).fit()
model1.summary()

Optimization terminated successfully.
         Current function value: 0.608111
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      460
Method:                           MLE   Df Model:                            1
Date:                Tue, 07 Sep 2021   Pseudo R-squ.:                 0.05740
Time:                        21:05:14   Log-Likelihood:                -280.95
converged:                       True   LL-Null:                       -298.05
Covariance Type:            nonrobust   LLR p-value:                 4.937e-09
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.1690      0.143     -8.169      0.000      -1.449      -0.889
famhist_Present     1.1690      0.203      5.751      0.000       0.771       1.567
===================================================================================
"""

In [ ]:
famhist_present_mean = df_bin["famhist_Present"].mean()

estimate_famhist_0 = model1.params["Intercept"] + (model1.params["famhist_Present"] * 0)
estimate_famhist_1 = model1.params["Intercept"] + (model1.params["famhist_Present"] * 1)

print("Probabilidad sin antecedentes familiares", round(logit_inv(estimate_famhist_0),5), "\nProbabilidad con antecedentes familiares", round(logit_inv(estimate_famhist_1),5))

Probabilidad sin antecedentes familiares 0.23704 
Probabilidad con antecedentes familiares 0.5


En relación a las preguntas:

1. La probabilidad de que un individuo con antecedentes familiares tenga una enfermedad coronaria es de un 50%.
2. La probabilidad de que un individuo sin antecedentes familiares tenga una enfermedad coronaria es de un 23.70%.
3. La diferencia es de un 26.3% a favor del caso con antecedentes familiares (considerando como positivo chd = 1).
4. Se replicará el modelo con smf.ols:

Nota: Se considerará solo el caso de "famhist_Present" como atributo en el model, y el caso de "famhist_Absent" será el valor del intercepto.


In [ ]:
model2 = smf.ols("chd ~ famhist_Present", data = df_bin).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    chd   R-squared:                       0.074
Model:                            OLS   Adj. R-squared:                  0.072
Method:                 Least Squares   F-statistic:                     36.86
Date:                Tue, 07 Sep 2021   Prob (F-statistic):           2.66e-09
Time:                        21:10:30   Log-Likelihood:                -294.59
No. Observations:                 462   AIC:                             593.2
Df Residuals:                     460   BIC:                             601.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.2370      0.028      8.489      0.000       0.182       0.292
famhist_Present     0.2630      0.043      6.071      0.000       0.178       0.348
==============================================================================
Omnibus:                      768.898   Durbin-Watson:                   1.961
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               58.778
Skew:                           0.579   Prob(JB):                     1.72e-13
Kurtosis:                       1.692   Cond. No.                         2.47
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Al observar el coeficiente de "famhist_Present" en el modelo de Regresión Lineal, se observa que es cerca de un 25% (22.5%) del coeficiente que presenta la misma variable en el modelo de Regresión Logística.

##Desafío 3: Estimación completa

1. Implemente un modelo que considere todos los regresores presentes en el DataFrame.
2. Depure el modelo manteniendo las variables con significancia estadística al 95%.
3. Compare los estadísticos de bondad de ajuste entre ambos.
4. Reporte de forma sucinta el efecto de las variables en el log-odds de tener una enfermedad coronaria.

Antes de continuar, se dropea columna "famhist_Absent".

In [ ]:
df_bin.drop(columns = "famhist_Absent", inplace = True)

In [ ]:
model3 = smf.logit("chd ~ sbp + tobacco + ldl + adiposity + typea + obesity + alcohol + age + famhist_Present", data = df_bin).fit()
model3.summary()

Optimization terminated successfully.
         Current function value: 0.510974
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      452
Method:                           MLE   Df Model:                            9
Date:                Tue, 07 Sep 2021   Pseudo R-squ.:                  0.2080
Time:                        21:10:47   Log-Likelihood:                -236.07
converged:                       True   LL-Null:                       -298.05
Covariance Type:            nonrobust   LLR p-value:                 2.055e-22
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -6.1507      1.308     -4.701      0.000      -8.715      -3.587
sbp                 0.0065      0.006      1.135      0.256      -0.005       0.018
tobacco             0.0794      0.027      2.984      0.003       0.027       0.132
ldl                 0.1739      0.060      2.915      0.004       0.057       0.291
adiposity           0.0186      0.029      0.635      0.526      -0.039       0.076
typea               0.0396      0.012      3.214      0.001       0.015       0.064
obesity            -0.0629      0.044     -1.422      0.155      -0.150       0.024
alcohol             0.0001      0.004      0.027      0.978      -0.009       0.009
age                 0.0452      0.012      3.728      0.000       0.021       0.069
famhist_Present     0.9254      0.228      4.061      0.000       0.479       1.372
===================================================================================
"""

Basándonos en el estadístico z y el punto de corte para una significancia estadística de un 95% (1.96), las variables que tienen un |z| > 1.96 son:

1. tobacco
2. ldl
3. typea
4. age
5. famhist_Present

Se realizará el nuevo modelo con estas variables independientes exclusivamente.

In [ ]:
model4 = smf.logit("chd ~ tobacco + ldl + typea + age + famhist_Present", data = df_bin).fit()
model4.summary()

Optimization terminated successfully.
         Current function value: 0.514811
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      456
Method:                           MLE   Df Model:                            5
Date:                Tue, 07 Sep 2021   Pseudo R-squ.:                  0.2020
Time:                        21:12:00   Log-Likelihood:                -237.84
converged:                       True   LL-Null:                       -298.05
Covariance Type:            nonrobust   LLR p-value:                 2.554e-24
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -6.4464      0.921     -7.000      0.000      -8.251      -4.642
tobacco             0.0804      0.026      3.106      0.002       0.030       0.131
ldl                 0.1620      0.055      2.947      0.003       0.054       0.270
typea               0.0371      0.012      3.051      0.002       0.013       0.061
age                 0.0505      0.010      4.944      0.000       0.030       0.070
famhist_Present     0.9082      0.226      4.023      0.000       0.466       1.351
===================================================================================
"""

Al observar ambos modelos, se tiene que:

1. R2 se mantiene prácticamente en la misma magnitud: ambos modelos mantienen la misma capacidad explicativa. Ahora bien, este indicador no castiga la cantidad de variables del modelo, por lo que no aumenta en el modelo depurado, a pesar de haber dejado 5 de las 9 variables presentes en el DataFrame.
2. Respecto a los indicadores z, estos aumentan (salvo en las variables typea y famhist_Present, casos en los que z disminuye) en el modelo depurado, respecto al modelo que incorpora todas las variables.



Para calcular el efecto de los log-odds de cada variable del modelo depurado, se calculan los promedios en cada caso (salvo famhist_Present que ya lo tiene calculado), y se generan los estimadores, para transformarlos posteriormente con logit_inv:

In [ ]:
tobacco_mean = df_bin["tobacco"].mean()
ldl_mean = df_bin["ldl"].mean()
typea_mean = df_bin["typea"].mean()
age_mean = df_bin["age"].mean()

estimate_tobacco = model4.params["Intercept"] + (model4.params["tobacco"] * tobacco_mean)
estimate_ldl = model4.params["Intercept"] + (model4.params["ldl"] * ldl_mean)
estimate_typea = model4.params["Intercept"] + (model4.params["typea"] * typea_mean)
estimate_age = model4.params["Intercept"] + (model4.params["age"] * age_mean)

print("Consumo promedio de tabaco:", round(tobacco_mean, 2), "\nProbabilidad (tabaco):", round(logit_inv(estimate_tobacco),5), "\nNivel LDL promedio:", round(ldl_mean, 0), "\nProbabilidad (LDL)", round(logit_inv(estimate_ldl),5), "\nPromedio puntaje personalidad:", round(typea_mean, 0), "\nProbabilidad (type A)", round(logit_inv(estimate_typea),5), "\nEdad promedio:", round(age_mean, 0), "\nProbabilidad (edad)", round(logit_inv(estimate_age),5), "\nProbabilidad (con antecedentes familiares)", round(logit_inv(estimate_famhist_1),5))

Consumo promedio de tabaco: 3.64 
Probabilidad (tabaco): 0.00212 
Nivel LDL promedio: 5.0 
Probabilidad (LDL) 0.00341 
Promedio puntaje personalidad: 53.0 
Probabilidad (type A) 0.01126 
Edad promedio: 43.0 
Probabilidad (edad) 0.01357 
Probabilidad (con antecedentes familiares) 0.5


Respecto a la incidencia de cada log-odd:

1. tobacco: la probabilidad de tener una enfermedad coronaria, para una persona que tiene un consumo de tabaco promedio de 3.64 (según los valores del DataFrame en análisis), es de un 0.21%
2. ldl: la probabilidad de tener una enfermedad coronaria, para una persona que tiene un nivel de lipoproteína de baja densidad promedio de 5.0 (según los valores del DataFrame en análisis), es de un 0.34%
3. typea: la probabilidad de tener una enfermedad coronaria, para una persona que tiene personalidad tipo A con un puntaje promedio de 53.0 (según los valores del DataFrame en análisis), es de un 1.13%
4. age: la probabilidad de tener una enfermedad coronaria, para una persona que tiene una edad promedio 43 años (según los valores del DataFrame en análisis), es de un 0.21%
5. famhist_Present: la probabilidad de tener una enfermedad coronaria, para una persona que tiene antecedentes familiares de este tipo de enfermedad (según los valores del DataFrame en análisis), es de un 0.21%

##Desafío 4: Estimación de perfiles

A partir del modelo depurado, genere las estimaciones en log-odds y posteriormente transfórmelas a probabilidades con ​inverse_logit​. Los perfiles a estimar son los siguientes:

1. La probabilidad de tener una enfermedad coronaria para un individuo con
características similares a la muestra.
2. La probabilidad de tener una enfermedad coronaria para un individuo con altos
niveles de lipoproteína de baja densidad, ​manteniendo todas las demás
características constantes.
3. La probabilidad de tener una enfermedad coronaria para un individuo con bajos
niveles de lipoproteína de baja densidad, ​manteniendo todas las demás
características constantes.

En relación a la pregunta 1, se toman los promedios ya calculados de cada variable, para posteriormente crear un estimador general y transformarlo con logit_inv:

In [ ]:
#1
estimate_general_prob = logit_inv(estimate_famhist_1) + logit_inv(estimate_tobacco) + logit_inv(estimate_ldl) + logit_inv(estimate_typea) + logit_inv(estimate_age)

#estimate_general = model4.params["Intercept"] + (model4.params["tobacco"] * tobacco_mean) + (model4.params["ldl"] * ldl_mean) + (model4.params["typea"] * typea_mean) + (model4.params["age"] * age_mean) + (model1.params["famhist_Present"] * famhist_present_mean)

print("Probabilidad de invididuo promedio del dataframe de tener una enfermedad coronaria:", estimate_general_prob)

Probabilidad de invididuo promedio del dataframe de tener una enfermedad coronaria: 0.5303574187588549


1. La probabilidad de que una persona con los niveles promedio en cada indicador tenga una enfermedad coronaria, es de un 53.04%

In [ ]:
#2 y #3
ldl_max = df_bin["ldl"].max()
ldl_min = df_bin["ldl"].min()

estimate_ldl_max = model4.params["Intercept"] + (model4.params["ldl"] * ldl_max)
estimate_ldl_min = model4.params["Intercept"] + (model4.params["ldl"] * ldl_min)

print(f"Probabilidad de tener enfermedad coronaria con altos niveles de LDL en sangre ({ldl_max}):", round(logit_inv(estimate_ldl_max),5), f"\nProbabilidad de tener enfermedad coronaria con bajos niveles de LDL en sangre ({ldl_min}):", round(logit_inv(estimate_ldl_min),5))

Probabilidad de tener enfermedad coronaria con altos niveles de LDL en sangre (15.33): 0.01865 
Probabilidad de tener enfermedad coronaria con bajos niveles de LDL en sangre (0.98): 0.00186


En relación a las preguntas 2 y 3:

2. La probabilidad de que una persona con altos niveles de ldl (lipoproteínas de baja densidad en sangre) tenga una enfermedad coronaria, es de un 1,87%
3. La probabilidad de que una persona con bajos niveles de ldl (lipoproteínas de baja densidad en sangre) tenga una enfermedad coronaria, es de un 0,19%

Manteniendo todo lo demás constante significa considerar las otras variables con el promedio, no dejarlas fuera..........

tb se podría haber usado .predict()

En el caso de arriba, no sé qué habría , poner uno con antecedentes y otro sin